In [5]:
from collections import defaultdict
from collections import defaultdict, Counter
import requests
import json
import time
from datetime import datetime, timedelta
from pandas import json_normalize
import pandas as pd
import calendar
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from concurrent.futures import ThreadPoolExecutor, as_completed

import asyncio
import aiohttp
import json


mes_passado = datetime.now() - timedelta(days=30)
mes_passado_inicio = mes_passado.strftime('%Y-%m-01 00:00:00')
mes_passado_fim = mes_passado.strftime('%Y-%m-31 23:59:59')

from dotenv import load_dotenv
import os

# Carregar variáveis do .env
load_dotenv()

token = os.getenv('TOKEN_API')

BASE_URL = 'http://10.0.100.128:5009'

In [7]:
# Endpoint da API
url = "https://assinante.nmultifibra.com.br/webservice/v1/cliente"

# Headers da requisição
headers = {
    'Authorization': token,
    'Content-Type': 'application/json',
    'ixcsoft': 'listar'
}

# Corpo da requisição
payload = {
    "qtype": "id_condominio",
    "query": "259",
    "oper": "=",
    "page": "1",
    "rp": "1000"
}

# Envio da requisição com headers
response = requests.post(url, json=payload, headers=headers)

# Processamento da resposta
if response.status_code == 200:
    data = response.json()
    clientes_ativos = [c for c in data.get("registros", []) if c.get("ativo") == "S"]
    
    print(f"Total de clientes ativos encontrados: {len(clientes_ativos)}\n")
    for cliente in clientes_ativos:
        print(f"ID: {cliente['id']} - Nome: {cliente['razao']} - CPF/CNPJ: {cliente['cnpj_cpf']}")
else:
    print(f"Erro na requisição: {response.status_code} - {response.text}")

Total de clientes ativos encontrados: 574

ID: 87000 - Nome: Vitor Lamano - CPF/CNPJ: 670.468.518-91
ID: 86947 - Nome: Luciana Santos de Sousa - CPF/CNPJ: 316.460.968-00
ID: 86732 - Nome: Elany Cristina Fernandes do Nascimento - CPF/CNPJ: 110.084.624-70
ID: 86688 - Nome: Leticia de Souza Laborão - CPF/CNPJ: 452.626.208-09
ID: 85629 - Nome: Marçal de Mancilha Junior - CPF/CNPJ: 307.075.438-43
ID: 85386 - Nome: Michael Esteves Santos - CPF/CNPJ: 338.082.388-01
ID: 85326 - Nome: Klisman Santos de Souza - CPF/CNPJ: 453.923.448-01
ID: 85150 - Nome: Alberto Rampazzo Filho - CPF/CNPJ: 091.910.988-82
ID: 84847 - Nome: Monique Fernanda Citadini - CPF/CNPJ: 384.348.918-16
ID: 84295 - Nome: Daiane Oliveira Vieira - CPF/CNPJ: 065.716.005-95
ID: 83862 - Nome: Fabio Joaquim Macedo Silva - CPF/CNPJ: 704.308.751-18
ID: 83740 - Nome: Joao Paulo Bastos da Costa - CPF/CNPJ: 035.480.003-55
ID: 83607 - Nome: Sueli da Rocha Queiroz - CPF/CNPJ: 227.131.428-30
ID: 83579 - Nome: Jeferson Francisco Rosa - CPF/C

In [14]:
# Headers
headers = {
    'Authorization': token,
    'Content-Type': 'application/json',
    'ixcsoft': 'listar'
}

# Requisição para buscar clientes do condomínio 259
url = "https://assinante.nmultifibra.com.br/webservice/v1/cliente"
payload = {
    "qtype": "id_condominio",
    "query": "410",
    "oper": "=",
    "page": "1",
    "rp": "1000"
}

response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    data = response.json()
    clientes_ativos = [
        {
            "id_cliente": c["id"],
            "nome": c["razao"],
            "cpf_cnpj": c["cnpj_cpf"],
            "telefone_comercial": c.get("telefone_comercial", ""),
            "telefone_celular": c.get("telefone_celular", ""),
            "whatsapp": c.get("whatsapp", "")
        }
        for c in data.get("registros", []) if c.get("ativo") == "S"
    ]

    # Salvar em JSON
    with open("clientes_ativosVivaCaucaia.json", "w", encoding="utf-8") as f:
        json.dump(clientes_ativos, f, ensure_ascii=False, indent=2)

    print(f"Clientes ativos salvos com contatos: {len(clientes_ativos)}")
else:
    print(f"Erro: {response.status_code} - {response.text}")

Clientes ativos salvos com contatos: 11


In [15]:
headers = {
    'Authorization': token,
    'Content-Type': 'application/json',
    'ixcsoft': 'listar'
}

# Carregar clientes ativos com contatos
with open("clientes_ativosVivaCaucaia.json", "r", encoding="utf-8") as f:
    clientes = json.load(f)

url_contrato = "https://assinante.nmultifibra.com.br/webservice/v1/cliente_contrato"

dados_finais = []

for cliente in clientes:
    payload = {
        "qtype": "id_cliente",
        "query": cliente["id_cliente"],
        "oper": "=",
        "page": "1",
        "rp": "1"
    }

    response = requests.post(url_contrato, json=payload, headers=headers)

    if response.status_code == 200:
        registros = response.json().get("registros", [])
        if registros and registros[0].get("status") == "A":
            dados_finais.append({
                "id cliente": cliente["id_cliente"],
                "nome cliente": cliente["nome"],
                "cpf": cliente["cpf_cnpj"],
                "plano": registros[0].get("contrato", ""),
                "telefone_comercial": cliente.get("telefone_comercial", ""),
                "telefone_celular": cliente.get("telefone_celular", ""),
                "whatsapp": cliente.get("whatsapp", "")
            })
    else:
        print(f"Erro cliente {cliente['id_cliente']}: {response.status_code}")

# Criar DataFrame e salvar em Excel
df = pd.DataFrame(dados_finais)
df.to_excel("clientes_com_contrato_ativoVivaCaucaia.xlsx", index=False)

print(f"Arquivo Excel gerado com {len(df)} clientes.")

Arquivo Excel gerado com 8 clientes.
